In [ ]:
# CHAINS! It usually combines LLM with prompts, and with this you can really use this to generate a great SEQUENCE OF EVENTS TOGETHER!
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read the local .env file
openai.api_key = os.environ['OPENAI_API_KEY'] # this is now the API key to use

testEnvString = os.environ['TEST_ENV'] # test to see if .env is working
print(testEnvString)

In [ ]:
# You can read several different kinds of inputs here!
# For example, let's load in a pandas dataframe